# CNN

# Explanation

### What is CNN? 
CNN is a Deep Learning algorithm that is mostly used for computer vision. Simply put,it slides a filter along the input features and computes an element-wise multiplication with a weight matrix. 
In our project, we want to use a one dimentional version of CNN. Similarly, we will slide a filter along our time-series data and elemtent-wise multiply a continues sub-series with a weight vector. The output is quiet "intuitive": we are essentially taking for example 5 days before day i, and averaging out their data to forecast the data of day i. 




In [27]:
import csv
inputfile = csv.reader(open('all_stocks_5yr.csv','r'))
stock_data = []
i = 0
for column in inputfile:
    if i == 10000:
        break
    i = i + 1
    place = column[1]
    stock_data.append(place)
stock_data.pop(0)
for i in range(len(stock_data)):
    stock_data[i] = float(stock_data[i])


In [28]:
from numpy import array
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

# define input sequence
raw_seq = stock_data

# choose a number of time steps
n_steps = 10
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(n_steps, n_features)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=1000, verbose=0)
# demonstrate prediction
raw_seq_sample = raw_seq[1990:-1]
x_input = array(raw_seq_sample)
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

ValueError: cannot reshape array of size 8008 into shape (1,10,1)